In [ ]:
import pandas as pd
from util.extract import extract_csv
from pathlib import Path

etapa = "chamada_regular"
ano = 2022
semestre = 1
arquivo_nome = f"{etapa}_sisu_{ano}_{semestre}.csv"

PATH = Path('./').parent.absolute()
arquivo_path = PATH / "sisu" / str(ano) / arquivo_nome
transformado_path = PATH / "transformado" / str(ano) / arquivo_nome
nome_tabela = arquivo_nome.replace('sisu_', '').replace('de_', '')
print(nome_tabela, "\nPATH ALVO: ",transformado_path)


[Dicionário de dados](https://dadosabertos.mec.gov.br/sisu/item/133-dicionario-de-dados)
para dataframes recentes.


In [ ]:
# dataframe de referencia, leve e com colunas padronizadas:
df_ref = pd.read_csv(
    PATH / 'reduzido' / 'reduzido_chamada_regular_sisu_2022_1.csv',
    sep="|",
    encoding="latin-1",
)


In [ ]:
try:
    df_alvo = extract_csv(ano, etapa, semestre)

except Exception as e:
    print(e)
    
df_alvo.shape

In [ ]:
# renomeia colunas
new_columns = {col: col.replace("NU_", "") for col in df_alvo.columns if "NU_" in col}
sg__columns = {col: col.replace("SG_", "") for col in df_alvo.columns if "SG_" in col}
st__columns = {col: col.replace("ST_", "") for col in df_alvo.columns if "ST_" in col}
no__columns = {
    col: col.replace("NO_", "NOME_") for col in df_alvo.columns if "NO_" in col
}
ind_columns = {
    "DT_NASCIMENTO": "DATA_NASCIMENTO",
    "TP_SEXO": "SEXO",
    "TP_MOD_CONCORRENCIA": "TIPO_MOD_CONCORRENCIA",
    "CO_ETAPA": "ETAPA",
    "CAMPUS": "NOME_CAMPUS",
    "CO_CAMPUS": "CODIGO_CAMPUS",
    "CO_IES": "CODIGO_IES",
    "IES": "NOME_IES",
    "CURSO": "NOME_CURSO",
    "CO_IES_CURSO": "CODIGO_CURSO",
    "CO_INSCRICAO_ENEM": "INSCRICAO_ENEM",
    "DS_GRAU": "GRAU",
    "DS_TURNO": "TURNO",
    "DS_MOD_CONCORRENCIA": "MOD_CONCORRENCIA",
}

l_cols = {
    "NOTACORTE": "NOTA_CORTE",
    "NOME_MUNUCIPIO_CAMPUS": "MUNICIPIO_CAMPUS",
    "NOME_INSCRITO": "INSCRITO",
}

new_columns.update(sg__columns)
new_columns.update(st__columns)
new_columns.update(no__columns)
new_columns.update(ind_columns)

df_alvo = df_alvo.rename(columns=new_columns).copy()
df_alvo = df_alvo.rename(columns=l_cols).copy()
df_alvo['CLASSIFICACAO'] = df_alvo['CLASSIFICACAO'].fillna(0).astype(float).astype(int)

In [ ]:
# remover colunas diferentes
df_alvo = df_alvo.copy().drop(["PERCENTUAL_BONUS"], axis=1)

cols_to_drop = [
    "DS_PERIODICIDADE",
    "TP_COTA",
]

if df_alvo.shape[1] > 53:
    df_alvo = df_alvo.copy().drop(
        cols_to_drop,
        axis=1,
    )

print(df_alvo.columns)
print(df_ref.columns)
# colunas diferentes
print("shape df_alvo:", df_alvo.shape)  # 53
print("shape df_ref:", df_ref.shape)  # 56
# colunas a mais: DS_PERIODICIDADE, TP_COTA, QT_VAGAS_CONCORRENCIA

In [ ]:
# verificar se ainda há colunas com nomes diferentes
columns_df1 = set(df_alvo.columns)
columns_df2 = set(df_ref.columns)

columns_only_in_df1 = sorted(list(columns_df1 - columns_df2))

columns_only_in_df2 = sorted(list(columns_df2 - columns_df1))

max_len = max(len(columns_only_in_df1), len(columns_only_in_df2))
data = {
    "colunas em df_alvo apenas": columns_only_in_df1
    + [""] * (max_len - len(columns_only_in_df1)),
    "colunas em df_ref apenas": columns_only_in_df2
    + [""] * (max_len - len(columns_only_in_df2)),
}
df_differences = pd.DataFrame(data)
display(df_differences)

In [ ]:
# converte colunas numéricas
for col in df_alvo.columns:
    if 'NOTA' in col or 'PESO' in col or "MEDIA_MINIMA" in col:
        df_alvo[col] = df_alvo[col].astype(str).str.replace(',', '.').astype(float)


In [ ]:
# adiciona coluna regiao
region_mapping = {
    "AC": "Norte",
    "AL": "Nordeste",
    "AM": "Norte",
    "AP": "Norte",
    "BA": "Nordeste",
    "CE": "Nordeste",
    "DF": "Centro-Oeste",
    "ES": "Sudeste",
    "GO": "Centro-Oeste",
    "MA": "Nordeste",
    "MT": "Centro-Oeste",
    "MS": "Centro-Oeste",
    "MG": "Sudeste",
    "PA": "Norte",
    "PB": "Nordeste",
    "PE": "Nordeste",
    "PI": "Nordeste",
    "PR": "Sul",
    "RJ": "Sudeste",
    "RN": "Nordeste",
    "RR": "Norte",
    "RS": "Sul",
    "SC": "Sul",
    "SE": "Nordeste",
    "SP": "Sudeste",
    "TO": "Norte",
    "RO": "Norte",
}
columns_reorder = [
    "ANO",
    "EDICAO",
    "ETAPA",
    "DS_ETAPA",
    "CODIGO_IES",
    "NOME_IES",
    "SIGLA_IES",
    "UF_IES",
    "CODIGO_CAMPUS",
    "NOME_CAMPUS",
    "REGIAO_IES",  # nova coluna
    "UF_CAMPUS",
    "MUNICIPIO_CAMPUS",
    "CODIGO_CURSO",
    "NOME_CURSO",
    "GRAU",
    "TURNO",
    "TIPO_MOD_CONCORRENCIA",
    "MOD_CONCORRENCIA",
    "QT_VAGAS_CONCORRENCIA", # dado apenas disponivel em 2022
    "PESO_L",
    "PESO_CH",
    "PESO_CN",
    "PESO_M",
    "PESO_R",
    "NOTA_MINIMA_L",
    "NOTA_MINIMA_CH",
    "NOTA_MINIMA_CN",
    "NOTA_MINIMA_M",
    "NOTA_MINIMA_R",
    "MEDIA_MINIMA",
    "CPF",
    "INSCRICAO_ENEM",
    "INSCRITO",
    "SEXO",
    "DATA_NASCIMENTO",
    "UF_CANDIDATO",
    "MUNICIPIO_CANDIDATO",
    "OPCAO",
    "NOTA_L",
    "NOTA_CH",
    "NOTA_CN",
    "NOTA_M",
    "NOTA_R",
    "NOTA_L_COM_PESO",
    "NOTA_CH_COM_PESO",
    "NOTA_CN_COM_PESO",
    "NOTA_M_COM_PESO",
    "NOTA_R_COM_PESO",
    "NOTA_CANDIDATO",
    "NOTA_CORTE",
    "CLASSIFICACAO",
    "APROVADO",
    "MATRICULA",
]

df_alvo["REGIAO_IES"] = df_alvo["UF_IES"].map(region_mapping)

if ano != 2022:
    columns_reorder.remove("QT_VAGAS_CONCORRENCIA")

df_alvo = df_alvo.reindex(columns=columns_reorder)

In [ ]:
df_alvo.to_csv(
    transformado_path,
    sep="|",
    encoding="utf-8",
    index=False,
)